In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model
import sklearn.svm
from sys import getsizeof
import sklearn.utils
import random
import sklearn.metrics

In [ ]:
%%time
#Sample ~s values from flights
n = 5800000
s = 500009
skip = np.random.choice(np.arange(1, n), n-s, replace = False)
flights = pd.read_csv(r"C:\Users\Garrett Williams\Desktop\flights.csv", skiprows = skip)
#airports = pd.read_csv(r"C:\Users\Garrett\Desktop\airports.csv")
#airlines = pd.read_csv(r"C:\Users\Garrett\Desktop\airlines.csv")

In [ ]:
%%time
#Feature preparation

#1hot encoding of airline
I = np.identity(14)
airlines_dict = dict(zip(flights['AIRLINE'].unique(), I) )
flights['airlines'] = flights['AIRLINE'].map(airlines_dict).astype(list)
#temp_airlines = np.array(flights['AIRLINE'].map(airlines_dict).astype(list))

#1hot encoding of Day of week
I = np.identity(7)
daysofweek = dict(zip(flights['DAY_OF_WEEK'].unique(), I) )
flights['daysofweek'] = flights['DAY_OF_WEEK'].map(daysofweek).astype(list)
#temp_dayofweek = np.array(flights['DAY_OF_WEEK'].map(daysofweek).astype(list))

#1hot encoding of Month
I = np.identity(12)
months = dict(zip(flights['MONTH'].unique(), I) )
flights['months'] = flights['MONTH'].map(months).astype(list)
#temp_months = np.array(flights['MONTH'].map(months).astype(list))


#1hot encoding of airports
I = np.identity(len(flights['ORIGIN_AIRPORT'].unique()))
origin = dict(zip(flights['ORIGIN_AIRPORT'].unique(), I))
flights['origin'] = flights['ORIGIN_AIRPORT'].map(origin).astype(list)
#temp_origin = np.array(flights['ORIGIN_AIRPORT'].map(origin).astype(list))

#1hot encoding of airports
I = np.identity(len(flights['DESTINATION_AIRPORT'].unique()))
depart = dict(zip(flights['DESTINATION_AIRPORT'].unique(), I))
flights['destination'] = flights['DESTINATION_AIRPORT'].map(depart).astype(list)
#temp_depart = np.array(flights['DESTINATION_AIRPORT'].map(depart).astype(list))


In [ ]:
%%time
#Map arrival delays onto 3 categories: early, 0 - 60 minutes late, greater than 60 minutes
arrival = []
delay_time = 60
for i in range(len(flights['ARRIVAL_DELAY'])):
    if flights['ARRIVAL_DELAY'][i] < 0:
        arrival.append(0)
    elif flights['ARRIVAL_DELAY'][i] >= 0 and flights['ARRIVAL_DELAY'][i] < delay_time:
        arrival.append(1)
    else:
        arrival.append(2)

arrival = np.array(arrival)

In [ ]:
%%time
#cancelled = np.array(flights['CANCELLED'])
labels_to_drop = ['origin', 'destination', 'SCHEDULED_DEPARTURE', 'SCHEDULED_TIME', 'DAY', 'YEAR','ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'ELAPSED_TIME', 'MONTH', 'DAY_OF_WEEK', 'AIRLINE', 'SCHEDULED_ARRIVAL', 'DEPARTURE_DELAY', 'DEPARTURE_TIME', 'TAXI_OUT','FLIGHT_NUMBER', 'TAIL_NUMBER', 'WHEELS_OFF', 'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN', 'CANCELLED', 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY', 'ARRIVAL_TIME', 'ARRIVAL_DELAY', 'DIVERTED']

In [ ]:
%%time
#Remove all NaN
flight = flights.drop(labels_to_drop, axis=1)
DATA = flight.dropna()

In [ ]:
flight.dtypes

In [ ]:
#Clears memory
flights = None 

In [ ]:
%%time
#Format data so its easier to handle with machine learning algorithms
N = np.round(len(DATA)/2)
X_train = np.array(DATA)[0:N]
X_test = np.array(DATA)[N:]

y = arrival[0:N]


X = []
for i in range(len(X_train)):
    X.append([x for y in X_train[i] for x in y]) 
X = np.array(X)


X2 = []
for i in range(len(X_test)):
    X2.append([x for y in X_test[i] for x in y]) 
X2= np.array(X2)

Data = None 
X_test = None
X_train = None

In [ ]:
%%time
for param in [1]:
    model = sklearn.linear_model.Ridge(alpha = param, fit_intercept=True)
    model = model.fit(X,y)
    values = model.predict(X2)
    print(sklearn.metrics.mean_squared_error(y, values))

In [ ]:
%%time
model = sklearn.linear_model.LogisticRegression(fit_intercept=True, solver = 'newton-cg', multi_class = 'multinomial')
model = model.fit(X,y)

In [ ]:
values = model.predict(X2)
sklearn.metrics.mean_squared_error(y, values)

In [ ]:
sklearn.metrics.mean_squared_error(y, [0]*len(y))

In [ ]:
sklearn.metrics.confusion_matrix(y, values)